In [1]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import seaborn as sns
import contextily as cx

In [33]:

slc_shapefile_path = 'zcta_data/slc_zcta.shp'
slc_gdf = gpd.read_file(slc_shapefile_path)
slc_gdf.to_crs(epsg=4326, inplace=True)
slc_gdf.head()

,ZCTA5,GEOID20,CLASSFP20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry
0,84070,84070,B5,G6350,S,19004717,0,+40.5771115,-111.8891029,"POLYGON ((-111.92159 40.57902, -111.92140 40.5..."
1,84108,84108,B5,G6350,S,187120680,381009,+40.8090403,-111.7275160,"POLYGON ((-111.84774 40.75849, -111.84726 40.7..."
2,84096,84096,B5,G6350,S,184045743,128519,+40.4870371,-112.0999829,"POLYGON ((-112.18765 40.51390, -112.18754 40.5..."
3,84129,84129,B5,G6350,S,17271536,0,+40.6535775,-111.9620632,"POLYGON ((-111.99152 40.63448, -111.99110 40.6..."
4,84084,84084,B5,G6350,S,17196254,0,+40.6213910,-111.9660607,"POLYGON ((-112.00622 40.62649, -112.00602 40.6..."


In [34]:
transit_df = pd.read_csv('./data/transit_dfs/TRX01_df.csv', low_memory=False, header=1)
# transit_df.head()

In [35]:
# Create a GeoSeries from the latitude and longitude data
transit_df['Point'] = gpd.GeoSeries([Point(xy) for xy in zip(transit_df['Longitude_ddeg'], transit_df['Latitude_ddeg'])])
transit_df.head()

,Timestamp_UTC,Latitude_ddeg,Longitude_ddeg,Elevation_m,Battery_Voltage_volts,Train_Box_Temperature_degC,Train_Top_Relative_Humidity_%,Train_Top_Temperature_degC,ES642_PM2.5_Concentration_ug/m3,ES642_Air_Flow_Rate_L/min,...,Month,Day_of_Year,Day_of_Month,Minute,Hour,Transit_ID,AQI,PM2.5_Category,Meso_West_Category,Point
0,2018-11-19 20:13:02,40.722687,-111.919289,1218.9,13.68,21.37,-9999.0,-9999.0,5.0,2.0,...,11,323,19,13,20,TRX01,Good,< 12.00,4.00 - 6.00,POINT (-111.91929 40.72269)
1,2018-11-19 20:14:10,40.722786,-111.918907,1219.4,13.68,21.37,-9999.0,-9999.0,5.0,2.0,...,11,323,19,14,20,TRX01,Good,< 12.00,4.00 - 6.00,POINT (-111.91891 40.72279)
2,2018-11-19 20:15:10,40.722919,-111.918556,1220.1,13.68,21.40,-9999.0,-9999.0,6.0,2.0,...,11,323,19,15,20,TRX01,Good,< 12.00,6.00 - 8.00,POINT (-111.91856 40.72292)
3,2018-11-19 20:16:22,40.722958,-111.918671,1221.8,13.65,21.40,-9999.0,-9999.0,5.0,2.0,...,11,323,19,16,20,TRX01,Good,< 12.00,4.00 - 6.00,POINT (-111.91867 40.72296)
4,2018-11-19 20:17:22,40.723015,-111.918709,1222.3,13.65,21.42,-9999.0,-9999.0,6.0,2.0,...,11,323,19,17,20,TRX01,Good,< 12.00,6.00 - 8.00,POINT (-111.91871 40.72301)


In [39]:
transit_gdf = gpd.GeoDataFrame(transit_df, geometry='Point', crs='epsg:4326')
transit_gdf.head()
joined = gpd.sjoin(transit_gdf, slc_gdf, how="left", op='within')
joined.head()

c:\Users\Raikou\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,Timestamp_UTC,Latitude_ddeg,Longitude_ddeg,Elevation_m,Battery_Voltage_volts,Train_Box_Temperature_degC,Train_Top_Relative_Humidity_%,Train_Top_Temperature_degC,ES642_PM2.5_Concentration_ug/m3,ES642_Air_Flow_Rate_L/min,...,index_right,ZCTA5,GEOID20,CLASSFP20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20
0,2018-11-19 20:13:02,40.722687,-111.919289,1218.9,13.68,21.37,-9999.0,-9999.0,5.0,2.0,...,32.0,84119.0,84119,B5,G6350,S,29817852.0,136726.0,+40.7002584,-111.9464247
1,2018-11-19 20:14:10,40.722786,-111.918907,1219.4,13.68,21.37,-9999.0,-9999.0,5.0,2.0,...,32.0,84119.0,84119,B5,G6350,S,29817852.0,136726.0,+40.7002584,-111.9464247
2,2018-11-19 20:15:10,40.722919,-111.918556,1220.1,13.68,21.40,-9999.0,-9999.0,6.0,2.0,...,32.0,84119.0,84119,B5,G6350,S,29817852.0,136726.0,+40.7002584,-111.9464247
3,2018-11-19 20:16:22,40.722958,-111.918671,1221.8,13.65,21.40,-9999.0,-9999.0,5.0,2.0,...,32.0,84119.0,84119,B5,G6350,S,29817852.0,136726.0,+40.7002584,-111.9464247
4,2018-11-19 20:17:22,40.723015,-111.918709,1222.3,13.65,21.42,-9999.0,-9999.0,6.0,2.0,...,32.0,84119.0,84119,B5,G6350,S,29817852.0,136726.0,+40.7002584,-111.9464247
